# Extractive Question Answering:

In [ ]:
pip install firebase-admin # Installing firebase

In [16]:
import json
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import time
# Firebase
import firebase_admin
from firebase_admin import credentials, firestore

In [17]:
#  S3 
s3_resource = boto3.resource('s3')
s3 = boto3.client('s3')

# DATA

In [5]:
data = pd.read_csv('recent_beds_df')

In [6]:
# Lowering text in area list
data['area'] = data['area'].str.lower()
area_list = list(set(data['area'].tolist()))
area_list.remove(area_list[0]) ### removing first value (nan)

### Defining the necessary functions. (Model endpoint functions, beds-data search function)

In [7]:
newline = '\n'
bold = '\033[1m'
unbold = '\033[0m'
def query_endpoint(encoded_text):
    endpoint_name = 'ENTER_YOUR_MODEL_ENDPOINT_NAME'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/list-text', Body=encoded_text)
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    answer = model_predictions['answer']
    return answer



def encode_bedtype (bert_answer):
    if (bert_answer.find('icu') == 0):
        bedtype = 1
    elif (bert_answer.find('venti') == 0):
        bedtype = 3
    elif (bert_answer.find('oxygen') == 0):
        bedtype = 2
    else:
        bedtype = 4
    return bedtype

def searchAndReply (area, bedtype):
    new = data.loc[data['area'] == area]
    
    hname = ""
    haddress = ""
    count = 0
    hnames = new['hospital_name'].tolist()
    haddresses = new['hospital_address'].tolist()
    btype = ""

    o = new['available_beds_with_oxygen'].tolist()
    ic = new['available_icu_beds_without_ventilator'].tolist()
    v = new['available_icu_beds_with_ventilator'].tolist()


    if (bedtype == 1):
        for i in range (0, len(ic)):
            if (ic[i] > 5):
                btype = " ICU "  
                count = ic[i]
                hname = hnames[i]
                haddress = haddresses[i]
                break
    elif (bedtype == 2):
        for i in range (0, len(o)):
            if (o[i] > 5):
                btype = " Oxygen " 
                count = o[i]
                hname = hnames[i]
                haddress = haddresses[i]
                break
    elif (bedtype == 3):
        for i in range (0, len(v)):
            if (v[i] > 5):
                btype = " Ventillator " 
                count = v[i]
                hname = hnames[i]
                haddress = haddresses[i]
                break
    elif (bedtype == 4):
        for i in range (0, len(ic)):
            if (o[i] > 5):
                btype = " Oxygen " 
                count = o[i]
                hname = hnames[i]
                haddress = haddresses[i]
                break


    if(count == 0):
        result = "No beds available"
    else:
        result = str(int(count))+btype+"beds are available in: "+hname+', '+haddress
    return result



In [8]:
searchAndReply("delhi", 1) # testing the search function

'100 ICU beds are available in: AIIMS Jhajjar, Tehsil Badli, Badsa, Haryana 124105, India'

In [9]:
result_dict = {}
requirement = "What kind of bed is needed?" # Defining the question for BERT, while we use tweet as the context.

In [10]:
# Firebase credentials file
cred = credentials.Certificate("python_credentials.json")
firebase_admin.initialize_app(cred)

In [11]:
# Initializing firebase
db = firestore.client()  
collection = db.collection('tweets')  
doc = collection.document('tweet')
collection2 = db.collection('answers')  

In [12]:
res = doc.get().to_dict()

In [13]:
print(res["status"], res["data"])

0 I need an oxygen bed in hyderabad


# Interacting with apps using firebase

In [ ]:
while (True):
    
    collection = db.collection('tweets')  
    doc = collection.document('tweet')
    res = doc.get().to_dict()
    collection2 = db.collection('answers')  
    status = res['status']
    tweet = res['data']
    question_context1 = []
    areaname = "null"
    
    if (status == "1"):
        question_context1 = [requirement, tweet]
        
        try:
            query_response = query_endpoint(json.dumps(question_context1).encode('utf-8'))
        except Exception as e:
            if e.response is not None and 'Error' in e.response and e.response.get('Error').get('Code') == 'ModelError':
                raise Exception(
                         "Backend scripts have been updated in February '22 to standardize response "
                         "format of endpoint response."
                         "Previous endpoints may not support verbose response type used in this notebook."
                         f"To use this notebook, please launch the endpoint again. Error: {e}."
                )
            else:
                raise
        try:
            answer = parse_response(query_response)
        except (TypeError, KeyError) as e:
            raise Exception(
                      "Backend scripts have been updated in February '22 to standardize response "
                      "format of endpoint response."
                       "Response from previous endpoints not consistent with this notebook."
                       f"To use this notebook, please launch the endpoint again. Error: {e}."
            )
        print (f"Inference:{newline}"
               f"Context: {question_context1[1]}{newline}"
                f"Question: {bold}{question_context1[0]}{unbold}{newline}"
                f"model answer: {bold}{answer}{unbold}{newline}")
        
         # Searching for areas for which we have beds information.

        for i in range (0, len(area_list)):    
            if (tweet.find(area_list[i]) > -1):
                print(area_list[i]+' found in the tweet: '+tweet)
                areaname = area_list[i]
        bedtype = encode_bedtype(answer)
        
        if (areaname == "null"):
            reply = "No beds available"
        else:
            reply = searchAndReply(areaname, bedtype)
            print(reply)
            
        # Update firebase
        collection.document('tweet').update({'status': '0'})
        collection2.document('answer').update({'data': reply})
    elif (status == "0"):
        pass
    else:
        print('error')
        
        

Inference:
Context: i need an icu bed in delhi
Question: What kind of bed is needed?
model answer: icu bed

delhi found in the tweet: i need an icu bed in delhi
100 ICU beds are available in: AIIMS Jhajjar, Tehsil Badli, Badsa, Haryana 124105, India
